## Fine tune a llama3 model with m-a-p/Code-Feedback

### Setup

In [ ]:
#!pip install -q transformers[torch] datasets
#!pip install -q bitsandbytes trl peft
#!pip install flash-attn --no-build-isolation

In [ ]:
# torch.cuda.empty_cache()

### Load Data + Preprocessing

In [ ]:
from datasets import load_dataset, DatasetDict

In [ ]:
raw_datasets = load_dataset("m-a-p/Code-Feedback")
raw_datasets

In [ ]:
# SPLIT data in train/test
indices_1 = range(0,100)
indices_2 = range(101,201)
dataset_dict = {
    "train": raw_datasets["train"].select(indices_1),
    "test": raw_datasets["train"].select(indices_2)
}
raw_dataset = DatasetDict(dataset_dict)
raw_dataset

In [ ]:
raw_dataset["train"][0]

#### Tokenizer

In [ ]:
from transformers import AutoTokenizer
from huggingface_hub import login

In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [ ]:
DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
import re
import random
from multiprocessing import cpu_count


def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

In [ ]:
column_names = list(raw_dataset["train"].features)
column_names


In [ ]:
raw_dataset = raw_dataset.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template",)

In [ ]:
train_data = raw_dataset["train"]
test_data = raw_dataset["test"]

for index in random.sample(range(len(raw_dataset["train"])), 3):
  print(f"Sample {index} of the processed training set:\n\n{raw_dataset['train'][index]['text']}")
  

### Model Definition

In [ ]:
# QloRa SFT
from transformers import BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
)

device_map = {"": torch.cuda.current_device()} if torch.cuda.is_available() else None
device_map


In [ ]:
model_kwargs = dict(
    #attnt_implementation=True,
    torch_dtype="auto",
    use_cache=False,
    device_map=device_map,
    quantization_config=quant_config.to_dict() 
)

### SFT Trainer

In [ ]:
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers import TrainingArguments

In [ ]:
output_dir = "../../model_saved/llama_code_feedback-8B"

In [ ]:
# based on config
training_args = TrainingArguments(
    fp16=True, # specify bf16=True instead when training on GPUs that support bf16
    do_eval=True,
    eval_strategy="epoch",
    gradient_accumulation_steps=128,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    learning_rate=2.0e-05,
    log_level="info",
    logging_steps=5,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    max_steps=-1,
    num_train_epochs=1,
    output_dir=output_dir,
    overwrite_output_dir=True,
    per_device_eval_batch_size=1, # originally set to 8
    per_device_train_batch_size=1, # originally set to 8
    save_strategy="no",
    save_total_limit=None,
    seed=42,
)

# based on config
peft_config = LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

In [ ]:
trainer = SFTTrainer(
        model=model_id,
        model_init_kwargs=model_kwargs,
        args=training_args,
        train_dataset=train_data,
        eval_dataset=test_data,
        dataset_text_field="text",
        tokenizer=tokenizer,
        packing=True,
        peft_config=peft_config,
        max_seq_length=tokenizer.model_max_length,
    )

In [ ]:
train_result = trainer.train()

### Saving Model

In [ ]:
metrics = train_result.metrics
max_train_samples = 100
metrics["train_samples"] = min(max_train_samples, len(train_data))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()
trainer.save_model(training_args.output_dir)

### Inference

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

output_dir = "../../model_saved/llama_code_feedback-8B"

tokenizer = AutoTokenizer.from_pretrained(output_dir)
model = AutoModelForCausalLM.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")

In [ ]:
import torch

# We use the tokenizer's chat template to format each message
messages = [
    {"role": "user", "content": "Write a Ruby code to convert a double-linked list to a single-linked list"},
]

DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '' }}\n{% endif %}\n{% endfor %}"
tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

# Prepare the messages for the model
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to("cuda")

# Ensure input_ids are valid
print("Input IDs:", input_ids)

# Inference with model output checks
outputs = model.generate(
        input_ids=input_ids,
        max_new_tokens=256,
        do_sample=True,
        temperature=1.0,  # Adjusted for stability
        top_k=50,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True
)

# Decode the generated output
decoded_output = tokenizer.batch_decode(outputs['sequences'], skip_special_tokens=True)[0]
print(decoded_output)
